PostId: 2019-04-30-222621
Title:DNNの情報理論的な論文を読む

Tishbyが良く研究しているっぽいので、その辺を中心に読んでいく。

## Deep Learning and the Information Bottleneck Principle

[arxiv1503.02406: Deep Learning and the Information Bottleneck Principle (2015)](https://arxiv.org/abs/1503.02406)

2015年。DNNの学習をIBの枠組みで見てみる。
入力と出力のMutual Information（以下MI）とか、hidden layerとhidden layerとhiddenの間のMIだとかを見ていく。
何かが分かったというよりは、IB的な枠組みで捉えなおすとこうなりますよね、という内容。

Cover and Thomasで出てくるRate distortion theoryとか十分統計量とかData proccessing inequalityとか、
関連する話題が多くてなかなか面白い。

## Opening the Black Box of Deep Neural Networks via Information

[arxiv1703.00810: Opening the Black Box of Deep Neural Networks via Information](https://arxiv.org/abs/1703.00810)

先ほどの2015年論文の続編。

### 対象としているデータ

3に具体的な相互情報量の計算方法が載っている。
30個の離散値に離散化して計算、とかなかなか面白い。

ただ教師データが何なのかがいまいち分からない。
12 binary inputsで、2Dのsphere(つまり円か？)の12 uniformly distributed pointsだと言っている。
Xはこの4096パターンがどうこう、と言っているが、良く分からない。
4096とは12 binaryという事で2の12乗っぽいが…

[ソースはあった](https://github.com/ravidziv/IDNNs)が、var_u.matという行列データを食わせているだけで、このデータの作り方は不明。
うーん。

Kazhdan 2003を参考にしているっぽいなぁ。
[Rotation Invariant Spherical Harmonic Representation of 3D Shape Descriptors (pdf)](https://www.cs.princeton.edu/~funk/sgp03.pdf) これか。
軽く読んだ。

3次元のモデルデータを、何らかのdescriptorにする事で検索とかしやすくする、という物。
回転不変なdescriptorにする為、いろいろやって64*64の二次元gird上の値にしている（Figure 4.)

ラベルデータとしてはViewpointのオブジェクトのラベルか。
これがKazhdan 2003らしい。

これをこの論文の3.1に書いてあるように変形している訳だ。
つまり、ラベルを、各グリッドが対象としているモデルのうち、平均よりスペクトルが大きいか？というのをラベルとしている。

…ん？どういう事だ？つまり適当な3Dモデル一つを選んで、4096通りの2値の教師データを作ったという事か？
めっちゃ少ないが大丈夫なのか。
そしてこのモデルに使ったデータが元のViewpointのどのオブジェクトかはどこかに書いておいてくれないと、このデータ作り直せないじゃないか。
再現実験でも同じ.matファイルを使いまわしているっぽいし、これは酷いなぁ。

ちなみに提供されてるスクリプトで再現できない、みたいな事を言っているブログがあった。
[Failure to replicate Schwartz-Ziv and Tishby](https://planspace.org/20180213-failure_to_replicate_schwartz-ziv_and_tishby/)

なお、さらに続きの実験している方はもうちょっとちゃんとした再現手順を書いている。
[github: Code for On the Information Bottleneck Theory of Deep Learning](https://github.com/artemyk/ibsgd)

ただ以上の理解を元にすれば、ようするに4096通りのXとそれに応じた0か1のYがある訳で、これらのテーブルデータから離散的な同時分布が分かる訳だ。
P(X, Y)が分かるのは理解した。

### Tの分布とはなんぞや？

問題はP(T, X)だな。TはXから一意に決まってしまうので、同時分布なんて無いように思えるが。（しいて言えばデルタ関数だが）

解説とコードを眺めていると、i番目のアクティベーションの値のそれぞれを別々の実現値として分布を考えているように見えるな。
それっておかしくないか？例えば3番目のレイヤーの一番上のニューロンと二番目のニューロンのactivationの値は別の変数で、
両者の関係はかなり謎だと思うのだが。
それを同じ確率変数の分布として計算した物にどういう意味があるんだろう？

うーん、意味は分からんがコードは自分の理解と同じ事をしているように見えるな。
これは結構何をしているか怪しいような？この論文は結構引用されたりバズったりしてた印象だが…

まあいい。あるレイヤーのそれぞれのアクティベーションの値をTiという確率変数のそれぞれの実現値とみなしてMIを計算している。

追記: 次のSaxeの追試の説明ではこの認識は間違ってそう。

### 学習の二つのフェーズの動画

論文には二つの動画を見ろ、とリンクが貼ってある。
二つ目のリンクはデッドリンクだった（なんなのだろう？）。

- [The optimization process in the Information Plane - symmetric rule](https://www.youtube.com/watch?v=P1A1yNsxMjc)

これは確かに主張の通りの綺麗な動画が取れているな。
動画の説明では色はどこのレイヤーかを表している、と言っている。

### 同一レイヤーの個々のニューロンの解釈は無意味

3.5の最後に、個々のニューロンの相関は無くなり、個々のニューロンのweightの解釈は無意味となる、と言っている。
どの辺からこれが言えているかはよく分からないが、
これがXとTの相互情報量で個々のニューロンのactivationを同一の確率変数の分布の結果のように考えるjustificationとなっている気もする。
説得力は微妙だが、根拠は一応ありそう（先に言ってよ、という気もするが）。

### 3.7 Focker-Planck equationってなんぞ？

拡散方程式の一種になる、という事で、制約条件的なのがついた形の知られている確率過程の問題になるらしい。
エクセンダールを見直したが同じ式は見つからなかった。
ただブラウン運動と似ているので、そういう感じのものなのだろう。

式11は衝撃的な内容で、こんな凄い事が本当に分かるの？と思ってしまうが、
Focker-Planck equationを知らないので本当かどうかも分からない。仕方ない。

感覚的にはそれぞれの場所でMaximum Entropyを解く感じになるので、式11的にそれぞれのエントロピーが下がるから学習しなくてはいけない分布もより単純になりそうな気はする。

この辺が本当だったらDeep LearningでDeepにする効果が分かったと言えちゃうよなぁ。
その割には実験の再現データの作り方とか皆からもっと突っ込み入りそうな物だが。

### 3.8 理論的なboundとの一致（が良く分からない）

とりあえず理論的なtの分布が出るのはいい。
で、これを収束した時の各hiddden layerの分布と比較する、なら話は分かるのだが、
この理論値にはベータというパラメータがある。
モデルの複雑さと答えの正確さのトレードオフみたいなのを調整する値。

で、このベータを、各hidden layerに最も近くなるように決める、で、決めたら近かった、と言っている。
うーん、うん？みたいな分からなさがあるな。

## On the Information Bottleneck Theory of Deep Learning

次はTishbyの論文はactiviationがtanhの時じゃないと再現しない、という、ある種の反論論文。

[https://openreview.net/forum?id=ry_WPG-A-](https://openreview.net/forum?id=ry_WPG-A-)

ちょっとTishbyの論文は特殊な前提をいろいろ置いていながらそれについてあまり考察せずに派手な結果だけ宣伝しがちに見えたので、
他の人はどう思ってるのかなぁ、というのを思って読んでみる。

なおTishbyサイドはSaxeのMIの推定には問題があると思っていて、たとえばこれを読め、みたいな事が本人のgithubのissueで貼られていた。

[Adaptive Estimators Show Information Compression in Deep Neural Networks](https://openreview.net/forum?id=SkeZisA5t7)

これも後で読みたい所。

### MIの計算手順についての解説が良い

2の所でガウス分布の入力を与えてニューロン一つの場合についての相互情報量などの考察があり、
この過程でbinningなどの影響が結構あるのでは無いか、という話が出ている。
これは自分も思っていた所で、Tishbyが本来自身の論文でやるべきだった議論では無いか？という気がする位重要な話に思う。

やっている作業についての細部についてちゃんと説明するって大切だよなぁ。
特にhidden layerとのMI、のような、何を求めているのかがそもそも曖昧な物については。

Tはそもそもdeterministicなのでそのままでは相互情報量は無限大だ、みたいな話があって、そうだよなぁ、と思うなど。
だから自分は同じレイヤーのそれぞれのニューロンのactivationを経験分布としているのかと思っていたのだが、誤解なのかしら？
この論文ではbinにするのは、binの中くらいしかわからなくなるようなノイズを加えている事に相当して、
そのおかげでMIが有限の値に収まる、という話が書いてある。

なお、自分が疑問に思ってたTを確率変数とするあたりの話については、Achiile, Soatto 2017で似たような話をしているっぽい。

Emergence of Invariance and Disentanglement in Deep Representations
[https://arxiv.org/abs/1706.01350](https://arxiv.org/abs/1706.01350)

これか。これもあとで読みたいな。

### Deep Linear Networkでの解析

activation無しでdeepなネットワークだと計算は単なるweightの積に落ちるので、
複雑な事を学習出来る根拠は無いのだけど解析的には扱いやすく、
しかも実際にトレーニングするといろいろ面白い現象も起こるらしい。
という事でそれをこのInformation Planeの計算でもやってみました、という話。

Deep Linear Netowrkの話はどこかで見かけた事はあるがちゃんと見た事は無かった。

この実験はAdvani, Saxe 2017と同じ設定らしい。
以下かな。

[High-dimensional dynamics of generalization error in neural networks](https://arxiv.org/abs/1710.03667)

generarization errorを分析する論文か。
VC dimensionとRademacher boundsって良く出てくるが全然知らんなぁ。

軽く実験のセットアップだけ眺めて戻ってくる。
NはXの次元らしい。さらに入力はP個ある。

教師のWをサンプリングする、という所の意味は良く分からないが、まぁ先に進もう。

そのあとにdifferential entropyが発散しちゃうのでガウシアンだとします、みたいな説明があって全く納得出来ない。
発散するのが正しい値なのになんで無根拠なノイズを足して有限の値にするのだろう？

追記: Appendix Cにその辺の議論がある。感想は後ろに足した。

### 4. Compressionってほんと？という話(が良く分からない)

Tishbyが主張しているCompresssionの話で、
条件付きエントロピーの解釈がちょっとおかしくないか？という話がある。
training runごとのTの違いとXの相互情報量とは何なのか？という。
こういうのはもっと議論されるべきと思うのだが、自分は良く理解出来ていない。

元の論文では別々のトレーニングの結果を平均してるんだっけ？読み直してみよう。
50回のランダムなweightの初期化でやったという記述はあるな。
確かにFigure 2もそう書いてある。平均している訳じゃないがinformation plane上に一緒に書いている訳か。

diffusionが定常分布になった後にそこからサンプルしたweightがH(X|T)を最大化すると言っているが、
この定常分布には異なるtrainingのセットアップから来るstochasticityを含んでいて、
それがH(X|T)を最大化するという根拠は無い、と言っている。

どうだろう？別に同じXを使っていれば、異なるtrainingのセットアップに対してもH(X|T)を最大化しうる、というのはありえそうな気もするが。
いまいちよく理解出来ていないなぁ。

ただfull batchでやってもcompression示すよ、は元の論文の論拠を否定するのには説得力あるな。
やはりいろいろ怪しい主張があるよねぇ。

### Appendix C: differential entorpyが発散しちゃう事とノイズの話

先ほど疑問に思ったノイズ足すってどうなの？という事について、Appendix Cはまるまるその件についての議論に充てている。良いね。

読んだ結果、やはり「なぜ発散する値こそが正しい値じゃないのか？」という疑問には全く答えていない。
ただ、この論文の批判対象が使っているbinに離散化するのと本質的には大差無い、というのは説得力がある。

どちらにせよ、厳密に計算すると発散する時に、発散しない為の何かをするなら、なぜそれがより本質的な値に近いと思うのか、
その理由を説明する必要があるよなぁ。

なんにせよこの論文の長いAppendixはちゃんとdiscussionしよう、という姿勢があって好感が持てる。

### Tishbyのopen reviewのコメントがめっちゃけんか腰

上記のちょくちょく分からん事について何かヒントが無いかなぁ、とOpen Reviewのコメントを見ていて、
[Tishby本人のコメントを発見](https://openreview.net/forum?id=ry_WPG-A-&noteId=ryJ6FjclG)。
めっちゃけんか腰やな(^^;

ただグラフを貼るだけで再現手順を示さないやり方は、やはり説得力に欠けるよなぁ。
一方Saxeのこの論文はJupyter Notebookとかも残していてずっと再現しようとする人への配慮がある。

ここまで読んだ印象としては、Tishbyの主張をすべて否定出来ているとは思わないのだけど、
たとえTishbyの主張が正しくても主張の仕方には問題があるな、と思う。

ただこうやってちゃんと反証が出て、それに反論を加えていく事で元の論文の主張がより深まっていくなら良い一歩だよな。

## Adaptive Estimators Show Information Compression in Deep Neural Networks

[Adaptive Estimators Show Information Compression in Deep Neural Networks](https://openreview.net/forum?id=SkeZisA5t7)

これはTishbyが反論に自身のissueで貼ってた論文。
MIの計算方法をもうちょっと深く考える必要があるよな、
と感じてきたので読んでみる事に。

### Adaptiveとbin

この論文の主な改善点はmutual informationの計算をadaptiveにした、という物。
adaptiveとは何を指しているか、という事を考えるには、まずbinの話に戻るのが良い。

もともとdifferential entropyは、中間レイヤーがconditionalにはdeterministicなので発散する。
という事でそもそもこの相互情報量という概念にどういう意味があるかは怪しい。

Tishbyはbinに離散化して計算する事で有限の値を得ていた。
このbinに離散化するのはnoiseを加えるのとだいたいは同じ事で、Saxeはこの両方を比較していた。
で、Tishbyはtanhのactiviationで起こる事と理論的な分析を合わせていろいろな主張をしていた。
SaxeはReLUとかに変えて追試したらそれらの主張は成り立たず、
さらにactivation無しでのdeepなlinearのnetworkでの学習や解析的な分析でさらにTishbyの主張を否定していた。

Saxeのやり方としては、ReLUの全値から、その最大値を調べて、その中を等分する、というbinの作り方をしていた。
これはTishbyのやり方の自然な延長といえる。
またノイズを加える場合も、全部のレイヤーに同じノイズを加えている。

だが、この論文の主な主張は、レイヤーごとにスケールに大きな意味の違いがある物に対して、一つのbinへの分割で分けるのはよく無い、という主張。
これはそれなりに説得力がある。

tanhでうまくいってたのはsaturateするからでは無くて、スケールが揃っていたから、という事になる。
ReLUなどは天井知らずなので他のレイヤー間で比較するのは適切じゃない。
これはtanhではcompressionが観測されるがReLUでは観測されない、というSaxeの結果をちゃんと説明出来ている。

で、この論文ではレイヤーごとにbinするのが良いのだが、
さらにこの考えを進めて、entropy based binというのを使っている。
ようするに各binで状態の数が同じになるようにbinに分ける、という物。
これはKohaviらが1996年に提唱してるやり方らしい。

これはTishbyやSaxeのやり方よりマシそう、というのは分かる。
そしてReLUがtanhなどに比べて難しいという事も理解出来る。

また、Saxeがkernel density estimationで求めた時にも起きてなかった説明も、
ノイズをどのレイヤーでも同じにしたからだ、という話をしていて、
レイヤーごとの分散のスケールに応じたノイズにしたらTishbyの報告した現象が再現「する場合もある」と言っている。

### 結果の解釈はなかなか難しい

さて、adaptiveにしたらReLUでもTishbyと同じようなダイナミクスになりました、めでたしめでたし、ならストーリーはわかりやすいのだが、
結果はそこまで明らかでも無い。

まず、最初のinitializationによって現象が再現したりしなかったりする。
そして平均を取ると、全くcompressionの傾向は見られなくなる(Fig 6.)。
これはむしろTishbyの主張を部分的には弱めている。
また、どちらにせよbin化の仕方とかinitializationとかにこんなにセンシティブでは、
何を主張しても偶然都合の良いのを見ているだけの可能性がある。
その難しさが明らかになったという気がするな。

ただ、L2 reguralizationを入れた結果はrobustで、これはいろいろと研究の余地がありそうな雰囲気を感じさせる。
これならやはりbatch norm入れたいよね。

generalizationとcompressの関係はいろいろ調べていて、
最後のレイヤーだけかろうじて相関がありそうだが、これも言うほど明らかでは無い。
むしろこの時点ではまだ関係はわからない、くらいが結論に思う。
ただ解いている問題が特殊なので、一般的に無いかどうかはよくわからない。
さらなる研究に期待、という感じかな。

### binの周りの解釈を考えてみよう

binの周りの処理にセンシティブな事を議論しているのだから、そもそもなんでbinにすべきなのか、という事をもっと議論すべきな気がする。
今回のadaptiveなbinも、そっちの方がマシなのは納得出来るにせよ、
そもそもなんで離散化するのかの理論的裏付けが無い。

TというのはXをgivenとするとstochasticじゃない訳だが、
ではなぜそんな物との相互情報量を考えるのか？という事を考えるべきと思う。

順番を逆に考えて、TとXの同時分布が存在しているとしよう。
この分布とactivationの値の関係ってなんだろう？
なんとなく最尤推定の値かMAP推定の値であってくれると嬉しいな。

点推定の値であるとするなら、この周りの分布というのは推計誤差でたぶんガウス分布だよな。
分布の幅というのは、推計の確かさを反映した物になるので、たぶんaccuracyを伝搬させた何かであるべきものだろう。
ただネットワークの特定のactivationがaccuracyにどう影響を与えるか、なんて全然わからないから計算不能だろうが。
これはSaxeなどがノイズを加えて計算している物に一致しているので、理論的に何をやっているかは理解出来る。

XとTの相互情報量とはなんだろう？

気分的には十分統計量みたいなのになって欲しい訳だが、
普通は十分統計量ってのは分布のファミリーがシータでパラメトライズされてて、
このシータとの相互情報量だよなぁ。

$$ I(\theta, X) = I(\theta, T(X))$$

一方で今回はXとT(X)の相互情報量を求めてる。これっておかしいよな。
本来求めたい物と違う物になってしまっているんじゃないか。

Xから分かるシータの不確実性というのは、T(X)からでも同じだけ分かるってのが定義な訳だ。
でも今回のケースでのシータとはなんぞや？

原理的にはもとの球面調和関数のスペクトルとなっているデータだったんだから、
このスペクトルを求める時に使ったオブジェクトのインデックスだよな。

この問題は特殊すぎるので、もっと普通のケースを考えた方が良い気がする。
画像の分類を考えよう。
シータというのは、究極的には画像に写ってる実際のオブジェクト、
より普通の設定ではオブジェクトの種類だな。
これはターゲットのラベルに近い物だが、原理的にはターゲットのラベルと完璧に一致しているかはよくわからない。

とにかく、オブジェクトの種類から画像の分布が生成される。
この時に、ピクセルとオブジェクトの種類の相互情報量と、
Tとオブジェクトの種類の相互情報量を考える、というのが本来の問題設定だよな。

これとTishbyの求めている物はだいぶ違う。
だからこの相互情報量とは一体なんなのか？Tの分布とは何を指しているのか？
はもうちょっと考えてみる必要がある気がする。
結局この事についてちゃんとしたロジックが無いと、アドホックなbinの良し悪しなんて議論出来ないのでは無いか。

### XとTの分布についてもう少し考えてみた

Xというのは、各実現値ごとにその出やすさの確率が本来はあるのだよな。
で、XをgivenとしたTの確率というのは、ある特定のXについてTの自信、みたいなのを表す訳だ。

それはタスクに依存する話で、ある実現値$$x_1$$の時はほぼ確実に答えを予測出来る。
この場合はTの分布はsteepになる。

一方である実現値$$x_2$$ではあんまり自信が無い。この場合はTはぼわっと広がる。

これで確かにXとTの同時分布を考える事は出来るが、
それと今求めている物は結構な距離があるよな。
この本当に求めたい物により近づく為にはどうするのが良いか？
というのが次の研究の方向性か。